This document is a Python exploration of this R-based document: http://m-clark.github.io/data-processing-and-visualization/ml.html.  It is intended for those new to modeling and related concepts.  Code is *not* optimized for anything but learning.  In addition, all the content is located with the main document, not here, so many sections may not be included.  I only focus on reproducing the code chunks and providing some useful context.

##### Preliminaries

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

# to suppress some output; I note where you can expect them.
import warnings
warnings.filterwarnings('ignore')

In [2]:
happy = pd.read_csv('../data/world_hapiness.csv')

def scale(x):
    return((x - np.nanmean(x, axis = 0)) / np.nanstd(x, axis = 0))

happy_processed = (
    happy[[
    'country', 
    'year', 
    'happiness_score',
    'democratic_quality',
    'generosity',
    'healthy_life_expectancy_at_birth',
    'log_gdp_per_capita'
    ]]
    .apply(
        lambda x: scale(x)
        if x.name in [
            'happiness_score',
            'democratic_quality',
            'generosity',
            'healthy_life_expectancy_at_birth'
        ]
        else x
    )
    .assign(year = happy.year - np.min(happy.year))
    .dropna()
)
    
happy_processed.head()

,country,year,happiness_score,democratic_quality,generosity,healthy_life_expectancy_at_birth,log_gdp_per_capita
7,Afghanistan,10,-1.625550,-1.950587,0.545034,-1.307413,7.500539
8,Afghanistan,11,-1.815967,-1.963219,0.314034,-1.333794,7.497038
9,Afghanistan,12,-1.431590,-1.998775,-0.687488,-1.360174,7.497755
18,Albania,10,-0.399795,0.442664,-0.517345,0.618363,9.302960
19,Albania,11,-0.669036,0.449130,-0.127154,0.657933,9.337532


# Machine Learning Overview

## Concepts

See the [main document](http://m-clark.github.io/data-processing-and-visualization/ml.html).  There is a notable overview regarding loss functions, bias-variance tradeoff, optimization and more.

## Techniques

Python really shines with ML, and there are many more options there relative to classical statistical methods.  For standard ML, `scikit-learn` has long been a popular tool.  For deep learning and other techniques, many other modules are available that can take you far.

### Regularized Regression

A starting point for getting into ML from the more inferential methods is to use *regularized regression*.  These are conceptually no different than standard LM/GLM types of approaches, but they add something to the loss function.

$$\mathcal{Loss} = \Sigma(y - \hat{y})^2 + \lambda\cdot\Sigma\beta^2$$

In the above, this is the same squared error loss function as before, but we add a penalty that is based on the size of the coefficients.  So, while initially our loss goes down with some set of estimates, the penalty based on their size might be such that the estimated loss actually increases. This has the effect of shrinking the estimates toward zero. Well, [why would we want that](https://stats.stackexchange.com/questions/179864/why-does-shrinkage-work)?  This introduces [bias in the coefficients](https://stats.stackexchange.com/questions/207760/when-is-a-biased-estimator-preferable-to-unbiased-one), but the end result is a model that will do better on test set prediction, which is the goal of the ML approach. The way this works regards the bias-variance tradeoff we discussed previously.  

The following demonstrates regularized regression using the just the basic `statsmodels` approach. It actually uses *elastic net*, which has a mixture of two penalties, one of which is the squared sum of coefficients (typically called *ridge regression*, as above) and the other is the sum of their absolute values (the so-called *lasso*). The `L1_wt` varies the relative amount of lasso and ridge regression (1 = lasso, 0 = ridge). `alpha` is the actual penalty amount.

In [3]:
happy_model_base = smf.ols(
  'happiness_score ~ democratic_quality + generosity + log_gdp_per_capita',
  data = happy_processed
).fit_regularized(alpha = .2, L1_wt = .25)

happy_model_base.params

Intercept            -0.066209
democratic_quality    0.466727
generosity            0.069540
log_gdp_per_capita    0.012013
dtype: float64

Compare to the original.

In [4]:
smf.ols(
  'happiness_score ~ democratic_quality + generosity + log_gdp_per_capita',
  data = happy_processed
).fit().params

Intercept            -5.703681
democratic_quality    0.134756
generosity            0.168902
log_gdp_per_capita    0.613600
dtype: float64

However, we wouldn't know what alpha or the weight parameter should be.  This is where scikit-learn can help us use cross validation to choose those.  First lets split our data into training and test.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = happy_processed.drop(columns=['happiness_score', 'country']) 
y = happy_processed[['happiness_score']]

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.33, 
    random_state = 1212
)

Next we create a grid of values to run models via K-fold cross-validation.

In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV


alphas = np.logspace(-10, -.5, 10)
lambdas = np.linspace(0, 1, 10)

tuned_parameters = [{'alpha': alphas, 'l1_ratio': lambdas}]
n_folds = 10

grid_search = GridSearchCV(
    ElasticNet(max_iter = 5e3, selection = 'random'), 
    tuned_parameters, 
    cv = n_folds, 
    refit = True,
    scoring = 'neg_mean_squared_error'
)

In [8]:
# to see available metrics for scoring
# from sklearn.metrics import SCORERS
# sorted(SCORERS.keys()) 

Now we can fit the model on the training data.

In [9]:
cv_results = grid_search.fit(X_train, y_train)  # you will likely get some convergence warnings

For reasons known only to the developers, rather than creating a simple directional flag, [all scikit learn optimizes in a maximizing fashion](https://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error), which means for something like mean squared error, it's actually dealing with negative values, and it doesn't bother to convert them to positive for you.

In [10]:
print(cv_results.best_score_)   
print(cv_results.best_params_)

-0.29778755058423045
{'alpha': 0.02782559402207126, 'l1_ratio': 0.0}


Now let's get our prediction with the best model to get our test set error.

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

pd.DataFrame({
    'RMSE_test': mean_squared_error(cv_results.predict(X_test), y_test), 
    'R2_test': r2_score(cv_results.predict(X_test), y_test)}, 
    index = [0]
)

,RMSE_test,R2_test
0,0.270871,0.659144


### Random Forests

A limitation of standard linear models, especially with many input variables, is that there's not a real automatic way to incorporate interactions and nonlinearities.  So we often will want to use techniques that do so.  To understand *random forests* and similar techniques (boosted trees, etc.), we can start with a simple decision tree.  To begin, for a single input variable (`X1`) whose range might be 1 to 10, we find that a cut at 5.75 results in the best classification, such that if all observations greater than or equal to 5.75 are classified as positive, and the rest negative. This general approach is fairly straightforward and conceptually easy to grasp, and it is because of this that tree approaches are appealing.

Now let’s add a second input (`X2`), also on a 1 to 10 range. We now might find that even better classification results if, upon looking at the portion of data regarding those greater than or equal to 5.75, that we only classify positive if they are also less than 3 on the second variable. The following is a hypothetical tree reflecting this.

<img src="../img/tree1.png" width="200"/>

This tree structure allows for both interactions between variables, and nonlinear relationships between some input and the target variable (e.g. the second branch could just be the same `X1` but with some cut value greater than 5.75).  Random forests randomly select a few from the available input variables, and create a tree that minimizes (maximizes) some loss (objective) function on a validation set.  A given tree can potentially be very wide/deep, but instead of just one tree, we now do, say, 1000 trees. A final prediction is made based on the average across all trees. 

We demonstrate the random forest again using `scikit-learn`.  As before, we can tune some of the parameters, in this case, the number of predictors to randomly select each tree and how many observations should be in each leaf.

In [12]:
from sklearn.ensemble import RandomForestRegressor


max_feature = [2, 3, X.shape[1]]
min_samples_leaf = [1, 5, 10]

tuned_parameters = [{'max_features': max_feature, 'min_samples_leaf': min_samples_leaf}]
n_folds = 10

grid_search = GridSearchCV(
    RandomForestRegressor(n_estimators = 500), 
    tuned_parameters, 
    cv = n_folds, 
    refit = True,
    scoring = 'neg_mean_squared_error'
)

In [13]:
cv_results = grid_search.fit(X_train, y_train) 

In [14]:
print(cv_results.best_score_)   
print(cv_results.best_params_)

-0.18394727678728975
{'max_features': 3, 'min_samples_leaf': 1}


In [15]:
pd.DataFrame({
    'RMSE_test': mean_squared_error(cv_results.predict(X_test), y_test), 
    'R2_test': r2_score(cv_results.predict(X_test), y_test)}, 
    index = [0]
)

,RMSE_test,R2_test
0,0.191651,0.782681


We see notable improvement over the regularized regression.

### Neural Nets

### Neural Networks

*Neural networks* have been around for a long while as a general concept in artificial intelligence and even as a machine learning algorithm, and often work quite well. In some sense, neural networks can simply be thought of as nonlinear regression. Visually however, we can see them as a graphical model with layers of inputs and outputs. Weighted combinations of the inputs are created and put through some function (e.g. the sigmoid function) to produce the next layer of inputs. This next layer goes through the same process to produce either another layer, or to predict the output, or even multiple outputs, which serves as the final layer. All the layers between the input and output are usually referred to as hidden layers. If there were a single hidden layer with a single unit and no transformation, then it becomes the standard regression problem.

<img src="../img/nnet.png" width="200"/>

As a simple example, we can run a simple neural network with a single hidden layer of up to 1000 units.  Since this is a regression problem, no further transformation is required of the end result to map it onto the target variable.  There are many tuning parameters I am not showing that could certainly be fiddled with as well. This is just an example that will run quickly with comparable performance to the previous. 

In [16]:
from sklearn.neural_network import MLPRegressor

In [17]:
hidden_layer_sizes = [500, 1000]  # size of single hidden layer
lr = ['constant', 'adaptive']     # type of learning rate
l2 = np.logspace(-4, -.1, 10)     # penalty parameter


tuned_parameters = [{
    'hidden_layer_sizes': hidden_layer_sizes, 
    'learning_rate': lr,
    'alpha': l2
}]

n_folds = 10

grid_search = GridSearchCV(
    MLPRegressor(max_iter = 1000), 
    tuned_parameters, 
    cv = n_folds, 
    refit = True,
    scoring = 'neg_mean_squared_error'
)

In [18]:
cv_results = grid_search.fit(X_train, y_train) # you get warnings but they can be ignored

In [19]:
print(cv_results.best_score_)   
print(cv_results.best_params_)

-0.2750095727981398
{'alpha': 0.0007356422544596414, 'hidden_layer_sizes': 500, 'learning_rate': 'adaptive'}


In [20]:
pd.DataFrame({
    'RMSE_test': mean_squared_error(cv_results.predict(X_test), y_test), 
    'R2_test': r2_score(cv_results.predict(X_test), y_test)}, 
    index = [0]
)

,RMSE_test,R2_test
0,0.280162,0.666469


## Interpreting the Black Box

See the [main document](http://m-clark.github.io/data-processing-and-visualization/ml.html).  Also the [lime module](https://pypi.org/project/lime/), as an example.

## Summary

Hopefully this has demystified the ML approach for you somewhat.  Nowadays it may take little effort to get to state-of-the-art results from even just a year or two ago, and which, for all intents and purposes, would be good enough both now and for the foreseeable future. Despite what many may think, it is not magic, but for more classically statistically minded, it may require a bit of a different perspective. 

## Machine Learning Exercises

### Exercise 1

Use the <span class="pack" style = "">ranger</span> package to predict the Google variable `rating` by several covariates.  Feel free to just use the standard function approach rather than all the tidymodels stuff if you want, but do use a training and test approach. You can then try the model again with a different tuning.  For the first go around, starter code is provided.

In [21]:
google = pd.read_csv('../data/google_apps.csv')
google.head()

,app,category,rating,reviews,size_in_MB,installs,type,price,content_rating,genres,last_updated,current_ver,android_ver,avg_sentiment_polarity,avg_sentiment_subjectivity
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19.0,"10,000+",Free,0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,NaN,NaN
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,0.152652,NaN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7,"5,000,000+",Free,0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,NaN,NaN
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25.0,"50,000,000+",Free,0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,NaN,NaN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,NaN,NaN


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [23]:
model_data = google[['installs', 'type', 'size_in_MB', 'content_rating', 'reviews', 'rating']].dropna()

In [24]:
X = pd.get_dummies(model_data, columns=['installs', 'type', 'content_rating']).drop('rating', axis=1)

In [25]:
X.shape

(7055, 29)

In [26]:
y = model_data[['rating']]

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.33, 
    random_state = 1212
)

model = RandomForestRegressor(n_estimators=1000)

In [27]:
model_fit = model.fit(X_train, y_train)

model_fit

RandomForestRegressor(n_estimators=1000)

In [28]:
from sklearn.metrics import mean_squared_error, r2_score

predictions = model.predict(X_test)


print(mean_squared_error(model.predict(X_train), y_train))
print(mean_squared_error(predictions, y_test))

0.044616887488486585
0.32286242261404796


### Exercise 2

Redo the above incorporating cross-validation to tune over hyper-parameters as we did in the previous demo.

### Exercise 3

Compare the result to a different method, either regularized regression or a simple neural net as demonstrated above, or feel free to try something else.